<a href="https://colab.research.google.com/github/MK316/Fall2023/blob/main/F23_StressQuiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stress placement practice (Fall F23)

In [ ]:
#@markdown [1] Install packages (getting ready)
%%capture
!pip install pandas ipywidgets gradio pyqrcode gtts pydub IPython

[File link](https://raw.githubusercontent.com/MK316/Fall2023/main/F23Phonology/F23_StressWordList.csv)

In [ ]:
#@markdown [2] File to read: from my Github (online)
csv_url = "https://raw.githubusercontent.com/MK316/Fall2023/main/F23Phonology/F23_StressWordList.csv"
#csv_url ="/content/yourfilename.csv"

☺️ If you want your file to gegenerate your audio, follow the steps:

1) Prepare your csv file (CSV UTF-8 쉼표로 분리된 파일, 확장자 sample.csv)  
2) upload it on colab (left panel: folder shape icon)  
3) Remove the second # below and replace your file name   
e.g., **#** csv_url = "/content/**yourfilename**.csv"  
=> csv_url = "/content/stress160.csv  
4) run the code

In [ ]:
# If you want to use your own csv file (ID, Words, POS columns should be in your csv file)

# csv_url = "/content/yourfilename.csv"

In [ ]:
#@markdown Play the quiz! (After running this code, you can download the csv file on the left panel 'my_wordlist.csv')
import pandas as pd
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load CSV from GitHub
# csv_url = 'YOUR_CSV_FILE_URL_ON_GITHUB'  # Replace with your GitHub CSV URL
df = pd.read_csv(csv_url)
df = df.dropna(how='all')

# Save the DataFrame to a CSV file
df.to_csv('/content/my_wordlist.csv', index=False, encoding='utf-8-sig')

# Quiz variables
score = 0
total_questions = 0

# Function to display a new question
def display_question():
    global total_questions
    random_row = df.sample().iloc[0]
    word = random_row['Words']
    pos = random_row['POS']  # Assuming 'POS' is the column name
    position = random_row['Position']

    # Update question text with POS information
    question_label.value = f'Word: {word} ({pos})'

    # Store the correct answer in a hidden widget
    correct_answer.value = position

    total_questions += 1
    result_label.value = ''  # Clear previous result

# Function to handle answer button click
def on_button_clicked(b):
    global score
    user_answer = b.description
    if user_answer == correct_answer.value:
        score += 1
        result_label.value = 'Good job!'
    else:
        result_label.value = f'The correct answer was: {correct_answer.value}'

# Create widgets
question_label = widgets.Label()
result_label = widgets.Label()
correct_answer = widgets.Label()  # This will be hidden
answer_buttons = [widgets.Button(description=p) for p in ['antepenultimate', 'penultimate', 'ultimate' ]]
continue_button = widgets.Button(description='Continue', button_style='success')  # Green color
finish_button = widgets.Button(description='Finish', button_style='danger')  # Red color

# Bind button events
for button in answer_buttons:
    button.on_click(on_button_clicked)

def continue_button_clicked(b):
    display_question()

continue_button.on_click(continue_button_clicked)

def finish_button_clicked(b):
    global score, total_questions
    clear_output()
    print(f'Your final Score is: {score}/{total_questions}')
    score = 0
    total_questions = 0

finish_button.on_click(finish_button_clicked)

# Layout for answer and control buttons
answer_buttons_box = widgets.HBox(answer_buttons)
control_buttons_box = widgets.VBox([continue_button, finish_button])
main_box = widgets.HBox([answer_buttons_box, control_buttons_box])

# Display widgets
display(question_label, result_label, main_box)

# Initial call to display the first question
display_question()


# Audio file to save

In [ ]:
#@markdown 🌀 Generate audio file from x to y (ID numbers)
import pandas as pd
from gtts import gTTS
from pydub import AudioSegment
from IPython.display import Audio
import io

url = csv_url

def generate_speech_with_pause(github_url, from_id, to_id):
    df = pd.read_csv(github_url)
    filtered_df = df[(df['ID'] >= from_id) & (df['ID'] <= to_id)]

    pause = AudioSegment.silent(duration=2000)  # 2000 milliseconds = 2 seconds
    combined = AudioSegment.empty()

    for word in filtered_df['Words']:
        # Use an in-memory buffer
        with io.BytesIO() as f:
            tts = gTTS(word)
            tts.write_to_fp(f)
            f.seek(0)
            word_audio = AudioSegment.from_file(f, format="mp3")
            combined += word_audio + pause

     # Export to an in-memory buffer
    buffer = io.BytesIO()
    combined.export(buffer, format="mp3")
    buffer.seek(0)

    return Audio(buffer.getvalue())

# Example usage
github_url = url
from_id = int(input('Start ID: '))
to_id = int(input('End ID: '))
audio = generate_speech_with_pause(github_url, from_id, to_id)
audio


# Audio files with POS information:

e.g., 'facilitate' when it's verb, 'opera' when it's noun, etc.

In [ ]:
#@markdown 🌀 Audio with POS info
import pandas as pd
from gtts import gTTS
from pydub import AudioSegment
from IPython.display import Audio
import io

url = csv_url  # Replace with your actual CSV URL

def generate_speech_with_pause(github_url, from_id, to_id):
    df = pd.read_csv(github_url)
    filtered_df = df[(df['ID'] >= from_id) & (df['ID'] <= to_id)]

    pause = AudioSegment.silent(duration=1000)  # 1000 milliseconds = 1 second
    combined = AudioSegment.empty()

    for index, row in filtered_df.iterrows():
        text_to_speak = f"{row['Words']} when it's {row['POS']}"
        with io.BytesIO() as f:
            tts = gTTS(text_to_speak)
            tts.write_to_fp(f)
            f.seek(0)
            word_audio = AudioSegment.from_file(f, format="mp3")
            combined += word_audio + pause

    # Export to an in-memory buffer
    buffer = io.BytesIO()
    combined.export(buffer, format="mp3")
    buffer.seek(0)

    return Audio(buffer.getvalue())

# Example usage
github_url = url
from_id = int(input('Start ID: '))
to_id = int(input('End ID: '))
audio = generate_speech_with_pause(github_url, from_id, to_id)
audio
